**Automação para atualizar as cotações da base de dados, assim gerando uma com as informações atualizadas**

OBS:
Para funcionar normalmente, baixar as seguintes bibliotecas:

*   Selenium
*   Webdriver
*   Utilizar o Jupyter Notebook, de preferência

Ao baixar o Webdriver, coloque o arquivo "chromedriver.exe" na mesma pasta que o arquivo "python.exe"

In [1]:
# Pegar as cotações do dia

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Dólar
navegador = webdriver.Chrome()
navegador.get('https://www.google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Euro
navegador.get('https://www.google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20292%2C87.')

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')

navegador.quit()

In [2]:
# Atualizar a base de dados

import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [8]:
# Recalcular os preços

# Atualizar as cotações
# Se a coluna moeda for igual a dólar, vai mudar a linha que tiver em dolar, e assim sucessivamente

tabela.loc[tabela["Moeda"] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# Preço de compra = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

#Preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

In [4]:
# Exportação da tabela atualizada

tabela.to_excel('Produto Atualizados.xlsx', index='False') # "index = 'False'" - não vai exibir o índice de numeração

In [5]:
# Base de dados atualizadas
tabela_nova = pd.read_excel("Produto Atualizados.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.076400,5076.349236,1.40,7106.888930
1,Carro Renault,4500.00,Euro,5.209681,23443.563969,2.00,46887.127938
2,Notebook Dell,899.99,Dólar,5.076400,4568.709236,1.70,7766.805701
3,IPhone,799.00,Dólar,5.076400,4056.043600,1.70,6895.274120
4,Carro Fiat,3000.00,Euro,5.209681,15629.042646,1.90,29695.181027
5,Celular Xiaomi,480.48,Dólar,5.076400,2439.108672,2.00,4878.217344
6,Joia 20g,20.00,Ouro,296.640000,5932.800000,1.15,6822.720000
